In [67]:
import numpy as np
import scipy
import implicit
import pandas as pd

print(np.__version__)
print(scipy.__version__)
print(implicit.__version__)

1.21.4
1.7.1
0.4.8


## 1) 데이터 준비와 전처리

In [68]:
import os
rating_file_path=os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/ratings.dat'
ratings_cols = ['user_id', 'movie_id', 'ratings', 'timestamp']
ratings = pd.read_csv(rating_file_path, sep='::', names=ratings_cols, engine='python', encoding = "ISO-8859-1")
orginal_data_size = len(ratings)
ratings.head()

,user_id,movie_id,ratings,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [69]:
# 3점 이상만 남김.
ratings = ratings[ratings['ratings']>=3]
filtered_data_size = len(ratings)

print(f'orginal_data_size: {orginal_data_size}, filtered_data_size: {filtered_data_size}')
print(f'Ratio of Remaining Data is {filtered_data_size / orginal_data_size:.2%}')

orginal_data_size: 1000209, filtered_data_size: 836478
Ratio of Remaining Data is 83.63%


In [70]:
# 영화 제목을 보기 위해 메타 데이터 읽음.
movie_file_path=os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/movies.dat'
cols = ['movie_id', 'title', 'genre'] 
movies = pd.read_csv(movie_file_path, sep='::', names=cols, engine='python', encoding='ISO-8859-1')
movies.head()

,movie_id,title,genre
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


In [71]:
# ratings에 있는 유니크한 영화 개수
print(ratings['movie_id'].nunique())

#rating에 있는 유니크한 사용자 수
print(ratings['user_id'].nunique())

3628
6039


In [72]:
# movies와 ratings 합한다.
data= pd.merge(movies, ratings)

In [73]:
data.head()

,movie_id,title,genre,user_id,ratings,timestamp
0,1,Toy Story (1995),Animation|Children's|Comedy,1,5,978824268
1,1,Toy Story (1995),Animation|Children's|Comedy,6,4,978237008
2,1,Toy Story (1995),Animation|Children's|Comedy,8,4,978233496
3,1,Toy Story (1995),Animation|Children's|Comedy,9,5,978225952
4,1,Toy Story (1995),Animation|Children's|Comedy,10,5,978226474


In [74]:
# 사용하는 컬럼만 남김.timestamp와 genre 없앰.
using_cols = [ 'user_id','title', 'ratings']
data = data[using_cols]
data.head(10)

,user_id,title,ratings
0,1,Toy Story (1995),5
1,6,Toy Story (1995),4
2,8,Toy Story (1995),4
3,9,Toy Story (1995),5
4,10,Toy Story (1995),5
5,18,Toy Story (1995),4
6,19,Toy Story (1995),5
7,21,Toy Story (1995),3
8,23,Toy Story (1995),4
9,26,Toy Story (1995),3


In [75]:
# 제목을 소문자로 바꿈.
data['title'] = data['title'].str.lower() 

In [76]:
# 가장 인기 있는 영화 30개 인기순으로 뽑기
movies_count = data.groupby('title')['user_id'].count()
movies_count.sort_values(ascending=False).head(30)

title
american beauty (1999)                                   3211
star wars: episode iv - a new hope (1977)                2910
star wars: episode v - the empire strikes back (1980)    2885
star wars: episode vi - return of the jedi (1983)        2716
saving private ryan (1998)                               2561
terminator 2: judgment day (1991)                        2509
silence of the lambs, the (1991)                         2498
raiders of the lost ark (1981)                           2473
back to the future (1985)                                2460
matrix, the (1999)                                       2434
jurassic park (1993)                                     2413
sixth sense, the (1999)                                  2385
fargo (1996)                                             2371
braveheart (1995)                                        2314
men in black (1997)                                      2297
schindler's list (1993)                                  2257
pr

In [77]:
# 내가 선호하는 영화 5 가지 골라서 ratings 추가
my_favorite = ['princess bride, the (1987)' , 'matrix, the (1999)' ,'shakespeare in love (1998)' ,
               'fargo (1996)' ,'toy story (1995)']

# 'sujin'이라는 user_id가 위 영화의 평점을 5점씩 주었다고 가정.
my_movielist = pd.DataFrame({'user_id': ['sujin']*5, 'title': my_favorite, 'ratings':[5]*5})

# user_id에 'sujin'이라는 데이터가 없다면
# 위에 임의로 만든 my_favorite 데이터를 추가. 
if not data.isin({'user_id':['sujin']})['user_id'].any():  
    data = data.append(my_movielist)   
    
data.tail(20) 

,user_id,title,ratings
836463,5074,"contender, the (2000)",4
836464,5087,"contender, the (2000)",4
836465,5100,"contender, the (2000)",4
836466,5205,"contender, the (2000)",4
836467,5304,"contender, the (2000)",4
836468,5333,"contender, the (2000)",4
836469,5359,"contender, the (2000)",5
836470,5405,"contender, the (2000)",4
836471,5475,"contender, the (2000)",5
836472,5602,"contender, the (2000)",3


## 모델 사용하기 위한 전처리

In [78]:
# 고유한 유저, 영화 제목을 찾아내는 코드
user_unique = data['user_id'].unique()
title_unique = data['title'].unique()

# 유저, 영화 제목을 indexing 하는 코드 idx는 index의 약자입니다.
user_to_idx = {v:k for k,v in enumerate(user_unique)}
title_to_idx = {v:k for k,v in enumerate(title_unique)}

In [79]:
# 인덱싱이 잘 되었는지 확인해 봅니다. 
print(user_to_idx['sujin'])    # 6040명의 유저 중 마지막으로 추가된 유저이니 6039이 나와야 함.
print(title_to_idx['toy story (1995)'])

6039
0


In [80]:
# indexing을 통해 데이터 컬럼 내 값을 바꾸는 코드

# user_to_idx.get을 통해 user_id 컬럼의 모든 값을 인덱싱한 Series를 구함.
# 혹시 정상적으로 인덱싱되지 않은 row가 있다면 인덱스가 NaN이 될 테니 dropna()로 제거

temp_user_data = data['user_id'].map(user_to_idx.get).dropna()
if len(temp_user_data) == len(data):   # 모든 row가 정상적으로 인덱싱되었다면
    print('user_id column indexing OK!!')
    data['user_id'] = temp_user_data   # data['user_id']을 인덱싱된 Series로 교체.
else:
    print('user_id column indexing Fail!!')

# temp_title_data을 통해 영화 제목 title 컬럼도 동일한 방식으로 인덱싱
temp_title_data = data['title'].map(title_to_idx.get).dropna()
if len(temp_title_data) == len(data):
    print('title column indexing OK!!')
    data['title'] = temp_title_data
else:
    print('title column indexing Fail!!')

data

user_id column indexing OK!!
title column indexing OK!!


,user_id,title,ratings
0,0,0,5
1,1,0,4
2,2,0,4
3,3,0,5
4,4,0,5
...,...,...,...
0,6039,1081,5
1,6039,2325,5
2,6039,2155,5
3,6039,587,5


## CSR matrix

In [82]:
data['ratings']

0    5
1    4
2    4
3    5
4    5
    ..
0    5
1    5
2    5
3    5
4    5
Name: ratings, Length: 836483, dtype: int64

In [85]:
from scipy.sparse import csr_matrix

num_user = data['user_id'].nunique()
num_movie = data['title'].nunique()

csr_data = csr_matrix((data.ratings, (data.user_id, data.title)), shape=(num_user, num_movie))
csr_data

<6040x3628 sparse matrix of type '<class 'numpy.int64'>'
	with 836483 stored elements in Compressed Sparse Row format>

## als_model = AlternatingLeastSquares 모델

In [86]:
from implicit.als import AlternatingLeastSquares
import os
import numpy as np
os.environ['OPENBLAS_NUM_THREADS']='1'
os.environ['KMP_DUPLICATE_LIB_OK']='True'
os.environ['MKL_NUM_THREADS']='1'

In [87]:
als_model = AlternatingLeastSquares(factors=100, regularization=0.01, use_gpu=False, iterations=30, dtype=np.float32)

In [88]:
csr_data_transpose = csr_data.T
csr_data_transpose

<3628x6040 sparse matrix of type '<class 'numpy.int64'>'
	with 836483 stored elements in Compressed Sparse Column format>

In [89]:
# 모델 훈련
als_model.fit(csr_data_transpose)

  0%|          | 0/30 [00:00<?, ?it/s]

## 내가 선호하는 5 개 영화 중 하나와 그 외의 영화 하나를 골라서 훈련된 모델이 예측한 나의 선호도를 파악

In [90]:
sujin, toy_story = user_to_idx['sujin'], title_to_idx['toy story (1995)']
sujin_vector, toy_story_vector = als_model.user_factors[sujin], als_model.item_factors[toy_story]

In [91]:
sujin_vector

array([-0.42656457,  0.31777644,  0.1513742 , -0.556501  , -0.23103666,
        0.6047816 , -0.44931737,  0.16323435, -0.08668626,  0.0438499 ,
       -0.03002334, -0.42183346,  0.3871217 ,  0.22069542,  0.36238432,
       -0.33757544,  0.5504052 , -0.33808863, -0.07103305, -0.1300755 ,
       -0.18920301,  0.38312995,  0.26576263, -1.0139203 , -0.45192865,
       -0.3817336 ,  0.09941719,  0.4873221 , -0.02909568, -0.31071985,
        0.00566387, -0.41492224,  0.19210179,  0.1876375 , -0.13852042,
        0.23828834,  0.03644371, -0.51000625,  0.5006811 ,  0.876707  ,
        0.26386306, -0.09867597,  0.85840935, -0.37591612, -0.288294  ,
        0.497554  , -0.11382608, -0.00836383, -0.64692384, -0.18606107,
       -1.0735682 ,  0.31821874,  0.23498839,  0.3753696 ,  0.4303767 ,
        0.48331976, -0.26000574,  0.96090955,  0.10332482, -0.38800198,
       -0.25927666, -0.5165015 , -0.11536215,  0.5032792 ,  0.79710865,
        0.5029438 , -0.02987443,  0.16686425,  0.05014873,  0.03

In [92]:
toy_story_vector

array([ 0.00555722,  0.02263323, -0.02353916, -0.02968935, -0.01520334,
        0.01757408, -0.00053933, -0.00574667,  0.00548441,  0.0220106 ,
       -0.00429273,  0.01995547,  0.01810647, -0.00340577,  0.03407633,
        0.00561883,  0.00448885, -0.01229365,  0.02503721,  0.01528916,
       -0.00336937,  0.03592196, -0.00555085, -0.04812089,  0.0332384 ,
       -0.00332842,  0.01486927,  0.02787071,  0.00259931, -0.01553671,
       -0.04251165,  0.0251295 ,  0.03565608,  0.03133354,  0.00625044,
       -0.014353  , -0.01363543, -0.04367847,  0.05719453,  0.03154012,
        0.00076677, -0.00804882,  0.03463992,  0.00116027,  0.03852678,
        0.00156331, -0.00627117,  0.03853635, -0.03195016,  0.01791366,
       -0.05414001, -0.03354618,  0.01583612,  0.00034717,  0.01025891,
        0.0237924 , -0.00894934,  0.02102612, -0.01300955,  0.00604654,
       -0.00063377, -0.0090828 , -0.01167537,  0.03344648, -0.00446137,
        0.05102944,  0.00550094,  0.03871956,  0.00348547, -0.00

In [93]:
np.dot(sujin_vector, toy_story_vector)

0.5734013

- back to the future에 대해 선호도가 낮다. 그에 비해 toystory는 선호도가 1이 아니라 0.5 정도로 나왔다.

In [94]:
back_to_the_future = title_to_idx['back to the future (1985)']
back_to_the_future_vector = als_model.item_factors[back_to_the_future]
np.dot(sujin_vector, back_to_the_future_vector)

0.16683765

## 내가 좋아하는 영화와 비슷한 영화 추천 받기

In [95]:
idx_to_title = {v:k for k,v in title_to_idx.items()}
# 좋아하는 영화 제목을 넣으면 비슷한 영화를 추천해주는 함수
def get_similar_title(title_name: str):
    title_id = title_to_idx[title_name]
    similar_title = als_model.similar_items(title_id)
    similar_title = [idx_to_title[i[0]] for i in similar_title]
    return similar_title

In [96]:
get_similar_title('toy story (1995)')

['toy story (1995)',
 'toy story 2 (1999)',
 'babe (1995)',
 'aladdin (1992)',
 "bug's life, a (1998)",
 'groundhog day (1993)',
 'pleasantville (1998)',
 'beauty and the beast (1991)',
 'lion king, the (1994)',
 "there's something about mary (1998)"]

In [97]:
get_similar_title('shakespeare in love (1998)')

['shakespeare in love (1998)',
 'groundhog day (1993)',
 'election (1999)',
 'elizabeth (1998)',
 'american beauty (1999)',
 'being john malkovich (1999)',
 'for ever mozart (1996)',
 'toy story (1995)',
 'four weddings and a funeral (1994)',
 'ghostbusters (1984)']

## 내가 가장 좋아할 만한 영화를 추천 받기

In [101]:
user = user_to_idx['sujin']
# recommend에서는 user*item CSR Matrix를 받습니다.
title_recommended = als_model.recommend(user, csr_data, N=20, filter_already_liked_items=True)
for i in title_recommended: 
    print("{} {:02.4f}".format(idx_to_title[i[0]], i[1]))

toy story 2 (1999) 0.5312
groundhog day (1993) 0.4736
babe (1995) 0.4721
silence of the lambs, the (1991) 0.3972
l.a. confidential (1997) 0.3834
terminator 2: judgment day (1991) 0.3704
shawshank redemption, the (1994) 0.3542
usual suspects, the (1995) 0.3470
bug's life, a (1998) 0.3400
pulp fiction (1994) 0.3340
sixth sense, the (1999) 0.3156
american beauty (1999) 0.2555
election (1999) 0.2488
star wars: episode iv - a new hope (1977) 0.2465
star wars: episode v - the empire strikes back (1980) 0.2462
reservoir dogs (1992) 0.2394
fugitive, the (1993) 0.2329
erin brockovich (2000) 0.2227
men in black (1997) 0.2221
being john malkovich (1999) 0.2204


-  나의 대해 추천해주는 영화에 toy stoty2가 있어서 놀랍다.

In [102]:
toy_story_2 = title_to_idx['toy story 2 (1999)']
explain = als_model.explain(user, csr_data, itemid=toy_story_2)
[(idx_to_title[i[0]], i[1]) for i in explain[1]]

[('toy story (1995)', 0.4237069367937053),
 ('shakespeare in love (1998)', 0.08217137413295683),
 ('princess bride, the (1987)', 0.02625659264964622),
 ('matrix, the (1999)', 0.0034721496511753145),
 ('fargo (1996)', -0.009824426579161528)]

- toy story 2는 toy story 와 많이 유사하여 추천을 한 것 같다.

# 회고

- 어려웠던 점 : als 모델을 선언하는 것을 조금 더 수정해보고 싶었다. 어떻게 바꾸면 더 좋은 유사도가 나올지 궁금했지만 어려워서 못했다.
- 알아낸 점 및 모호한 점 : movie id에서 라벨인코딩처럼 하는 것을 하는 이유에 대해 궁금하다. 그것을 하지 않으면 CSR에서 오류가 나오는 것같다.
- 노력한 점 : 백터 내적 수치에 대해서 어떤 것이 높게 나올지 낮게 나올지에 대해 찾는 노력을 했다. 
- 자기다짐 : 추천시스템에 SVD에 대해서 자세히 공부해야겠다고 느꼈다!